In [0]:
# ! sudo apt install openjdk-8-jdk
# ! sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java 
# ! pip install language-check -qq
# ! pip install pycontractions -qq

In [0]:
! pip install chatterbot 
! pip install chatterbot_corpus

In [0]:
! pip install --upgrade chatterbot 
! pip install --upgrade chatterbot_corpus

In [0]:
# Import statements
import pandas as pd
import numpy as np
import pprint as pp
import json
from pandas.io.json import json_normalize
import re
from timeit import default_timer

# Preprocessing
# from pycontractions import Contractions

# Tokenization imports
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

# Puncuation & lower case
import string #punctuation removal

# Stop words
from nltk.corpus import stopwords

# Stemming
from nltk.stem.snowball import SnowballStemmer

# Lemmatizer
from nltk.stem.wordnet import WordNetLemmatizer

# POS tagging
from nltk.corpus import wordnet

# NER
import nltk, nltk.tag, nltk.chunk 
import spacy
import pprint as pprint
from gensim.summarization import summarize 
from collections import Counter 
import en_core_web_sm # CNN gets loaded in, sees what words depends on each other, POS tagging, entity recognition 
from spacy import displacy # Visualize NER

# Chatterbot
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

# Data extraction

In [0]:
# Load JSON file into dataframe
data = pd.read_json('/content/drive/My Drive/frames.json')
df = pd.DataFrame(data)

In [0]:
df.head() # Looks like turns and labels are nested dictionaries

,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."


In [0]:
df['id'].nunique() # There are 1369 unique ID for the dialogue

1369

In [0]:
# labels is a nested dictionary
labels_df = json_normalize(df['labels'])

In [0]:
labels_df.head()

,userSurveyRating,wizardSurveyTaskSuccessful
0,4.0,True
1,3.0,True
2,2.0,False
3,5.0,True
4,5.0,True


`userSurveyRating` - value that represents the user’s satisfaction with the Wizard’s service, ranging from 1 – complete dissatisfaction to 5 – complete satisfaction

`wizardSurveyTaskSuccessful` - boolean which is true if the wizard thinks at the end of the dialogue that the user’s goal was achieved

In [0]:
# Attempted json_normalize but kept getting the error: 'list' object has no attribute 'values'
# This happens because of list enclosing each of the dictionary.
#   - Multiple dictionaries with matching keys enclosed by a list.
#   - A dictionary with keys as columns and values in the form of list.

# Thus we try another approach
text_list = []
for item in df['turns']:
  text_list.append(item)
turns_df = pd.DataFrame(text_list)

In [0]:
turns_df[476:478] # Every row is a dialogue 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
476,{'text': 'So I live in Long Beach. What do you...,"{'db': {'result': [], 'search': []}, 'text': '...",{'text': 'just me and it can be between august...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'yes that would be great', 'labels': ...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'I thought you said it fit my dates, ...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'Not really, is that all you have for...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'Yes, next on my list is San Francisc...",{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': '10 days should be good', 'labels': {...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'I haven't set a budget for myself, I...","{'db': {'result': [], 'search': []}, 'text': '...",{'text': 'I like the idea of being in the hear...,"{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'economy', 'labels': {'acts': [{'args...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'nice! that's a great deal!', 'labels...","{'db': {'result': [], 'search': []}, 'text': '...",{'text': 'have past guests been satisfied ther...,"{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'ok, so decent reviews, for the price...","{'db': {'result': [], 'search': []}, 'text': '...",{'text': 'yup I don't have to leave exactly th...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'cool, tell me more!', 'labels': {'ac...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'any free wifi?', 'labels': {'acts': ...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'cool thanks!', 'labels': {'acts': [{...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'Last place is Hamburg, do you have a...",{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'I don't mind, is of them in the hear...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'That sounds like good hotel, how muc...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'i prefer economy', 'labels': {'acts'...","{'db': {'result': [], 'search': []}, 'text': '...",{'text': 'Does this hotel have better reviews ...,"{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'Ok, Hamburg was the destination I wa...","{'db': {'result': [], 'search': []}, 'text': '...",{'text': 'I'll take a chance there wih the 10 ...,"{'db': {'result': [], 'search': []}, 'text': '..."
477,"{'text': 'Get me on a flight to San Antonio, s...","{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'Kyoto. Get me the best hotel also', ...","{'db': {'result': [], 'search': []}, 'text': '...",{'text': 'I want to keep it under $5500 if pos...,"{'db': {'result': [], 'search': []}, 'text': '...","{'text': 'Yes all by my lonesome', 'labels': {...",{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'Its gotta be San Antonio man, damn O...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [0]:
turns_df.replace(np.nan,'',inplace=True) # Replace all null values

In [0]:
turns_df_T = turns_df.transpose() # Transposed the dataframe to have proper conversation turns

In [0]:
# Iterates through the data frame and gets all values with a text key
convo_text = ''
for i in turns_df_T:                                 # 1369 columns
  for j in range(0,48):                              # 48 rows
    try:
      get_text = turns_df_T[i][j].get('text')        # Gets the value from the corresponding key
      convo_text += get_text + '\n'                   # Concatenates all text
    except AttributeError:
      convo_text += '*'                               # Concatenates '' if there is no text key - used for switching conversations

In [0]:
pp.pprint(convo_text[0:1000])

("I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, "
 '2016 for 8 adults. I have a tight budget of 1700.\n'
 'Hi...I checked a few options for you, and unfortunately, we do not currently '
 'have any trips that meet this criteria.  Would you like to book an alternate '
 'travel option?\n'
 'Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 '
 'adults. For this trip, my budget would be 1900.\n'
 'I checked the availability for this date and there were no trips available.  '
 'Would you like to select some alternate dates?\n'
 'I have no flexibility for dates... but I can leave from Atlantis rather than '
 'Caprica. How about that?\n'
 'I checked the availability for that date and there were no trips available.  '
 'Would you like to select some alternate dates?\n'
 "I suppose I'll speak with my husband to see if we can choose other dates, "
 "and then I'll come back to you.Thanks for your help\n"
 '*****************************************He

In [0]:
with open('convo_text.txt', "w") as text_file:
    text_file.write(convo_text)

# Data Preprocessing

In [0]:
# This is testing with part 1 - 10% of data
with open('/content/drive/My Drive/contraction_data_parts/expand_convo_text_1.txt', 'r') as file:
    convo_all = file.read()

In [0]:
# Conversation sample
pp.pprint(convo_all[0:1000]) 

('["I would like to book a trip to Atlantis from Caprica on Saturday, August '
 '13, 2016 for 8 adults. I have a tight budget of 1700.\\nHi...I checked a few '
 'options for you, and unfortunately, we do not currently have any trips that '
 'meet this criteria.  Would you like to book an alternate travel '
 'option?\\nYes, how about going to Neverland from Caprica on August 13, 2016 '
 'for 5 adults. For this trip, my budget would be 1900.\\nI checked the '
 'availability for this date and there were no trips available.  Would you '
 'like to select some alternate dates?\\nI have no flexibility for dates... '
 'but I can leave from Atlantis rather than Caprica. How about that?\\nI '
 'checked the availability for that date and there were no trips available.  '
 'Would you like to select some alternate dates?\\nI suppose I will speak with '
 'my husband to see if we can choose other dates, and then I will come back to '
 'you.Thanks for your help\\n**************************************

In [0]:
# Let's break this text into separate lists for each conversation 
list_convos = [convo.split('\n') for convo in convo_all.split('*') if convo_all] # Conversation delimited by *
list_convos = [convo for convo in list_convos if convo != ['']]                  # Remove empty conversation

In [0]:
# Now we need to break each conversation by separate statements
pp.pprint(list_convos[10])

['I would like to get away from Monday, August 15, 2016 to Wednesday, August '
 '31, 2016. My budget is $3200 and I would leave from Detroit.\\nAnd what '
 'destination would you like to travel to?\\nAnywhere, I am looking for an '
 'adventure. What do you have?\\nHow about Mexico or Porto Alegre?\\nlet us '
 'look at both\\nMexico is unavailable.  Porto Alegre is available, but you '
 'would have to leave from Santos.\\nCan you please suggest some destinations '
 'that are available? I am leaving from Detroit.\\nI have no departures '
 'available from Detroit. for these dates.  Would you like to make a date '
 'change?\\nAnything between August 15 and August 31?\\nI am sorry, but we do '
 'not have any departures from Detroit available for these dates for the '
 'budget you provided.  Would you like to adjust your budget for this '
 'trip?\\nNo I cannot. Thank you for your help\\n']


In [0]:
test_convo_1 = list_convos[0][0].split('\\n')

In [0]:
sep_convos = []
for items in list_convos:
  sep_convos.append([])
  for item in items:
    item = item.split('\\n')
    sep_convos[-1].append(item)

In [0]:
len(sep_convos)

120

In [0]:
sep_convos[0]

[['["I would like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.',
  'Hi...I checked a few options for you, and unfortunately, we do not currently have any trips that meet this criteria.  Would you like to book an alternate travel option?',
  'Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.',
  'I checked the availability for this date and there were no trips available.  Would you like to select some alternate dates?',
  'I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that?',
  'I checked the availability for that date and there were no trips available.  Would you like to select some alternate dates?',
  'I suppose I will speak with my husband to see if we can choose other dates, and then I will come back to you.Thanks for your help',
  '']]

In [0]:
sep_convos_remove = []
for convo in sep_convos:
  sep_convos_remove.append([])
  for statement in convo:
    statement = re.sub(r"[\[\]]", "", statement)
    sep_convos_remove[-1].append(item)

len(sep_convos[0])
flat_list = [item for sublist in sep_convos[0] for item in sublist]

flat_list = [item for sublist in l for item in sublist]
flat_list = []
for convo in sep_convos:
  for sublist in convo:
      for item in sublist:
          flat_list.append(item)

In [0]:
flat_list = []
for convo in sep_convos:
  for sublist in convo:
      for item in sublist:
          flat_list.append(item)

In [0]:
len(flat_list)

1938

## Tokenization

In [0]:
nltk.download('punkt') # Punkt sentence tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
#split into sentences; sent_tokenize doesn't do the job properly
tokenized_sent = sent_tokenize(list_convos[0][0])
print(tokenized_sent[0:5])

['["I would like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults.', 'I have a tight budget of 1700.\\nHi...I checked a few options for you, and unfortunately, we do not currently have any trips that meet this criteria.', 'Would you like to book an alternate travel option?\\nYes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults.', 'For this trip, my budget would be 1900.\\nI checked the availability for this date and there were no trips available.', 'Would you like to select some alternate dates?\\nI have no flexibility for dates... but I can leave from Atlantis rather than Caprica.']


In [0]:
len(test_convo_1) # has 8 sentences

8

In [0]:
#split into words
tokenized_words=[]
tokenized_words.extend(word for word in word_tokenize(str(test_convo_1)))
pp.pprint(tokenized_words[-10:])

['back', 'to', 'you.Thanks', 'for', 'your', 'help', "'", ',', "''", ']']


## Punctuation & Lower Case

In [0]:
string.punctuation = string.punctuation + '``' + '...'

In [0]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~``...``...'

In [0]:
# Removes punctuation
no_punct = [token for token in tokenized_words if token not in string.punctuation] 
print(no_punct[-10:])

['I', 'will', 'come', 'back', 'to', 'you.Thanks', 'for', 'your', 'help', "''"]


In [0]:
no_punct = [word.strip('\'') for word in no_punct] # Pesky ' attatched to word
no_punct = [word.replace('.',' ') for word in no_punct] # Pesky . attatched to word

In [0]:
# Lowercase text
data_lower = [word.lower() for word in no_punct]
print(data_lower[-10:])

['i', 'will', 'come', 'back', 'to', 'you thanks', 'for', 'your', 'help', '']


## Stop Words

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
stop_words = stopwords.words('english')
add_stop_words = ['would','could']
stop_words += add_stop_words
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [0]:
filtered_word = []
for word in data_lower:
  if word not in stop_words:
    filtered_word.append(word)

In [0]:
filtered_word

['like',
 'book',
 'trip',
 'atlantis',
 'caprica',
 'saturday',
 'august',
 '13',
 '2016',
 '8',
 'adults',
 'tight',
 'budget',
 '1700',
 'hi',
 'checked',
 'options',
 'unfortunately',
 'currently',
 'trips',
 'meet',
 'criteria',
 'like',
 'book',
 'alternate',
 'travel',
 'option',
 'yes',
 'going',
 'neverland',
 'caprica',
 'august',
 '13',
 '2016',
 '5',
 'adults',
 'trip',
 'budget',
 '1900',
 'checked',
 'availability',
 'date',
 'trips',
 'available',
 'like',
 'select',
 'alternate',
 'dates',
 'flexibility',
 'dates',
 'leave',
 'atlantis',
 'rather',
 'caprica',
 'checked',
 'availability',
 'date',
 'trips',
 'available',
 'like',
 'select',
 'alternate',
 'dates',
 'suppose',
 'speak',
 'husband',
 'see',
 'choose',
 'dates',
 'come',
 'back',
 'you thanks',
 'help',
 '']

### Emoji removal

In [0]:
for word in filtered_word[:]:       #makes a copy of the list words and then iterates over that copy. Then, modifies the original list.
    if word.endswith('_face'): 
        filtered_word.remove(word) 

## Stemming

In [0]:
stemmer = SnowballStemmer('english', ignore_stopwords=True) # Already removed stopwords

In [0]:
stemmed_words = []

for words in filtered_sent:
  
  stemmed_words.append(stemmer.stem(words)) 

  print('Words '+words+' - stemmer:'+stemmer.stem(words))

In [0]:
filtered_words = []
for word in stemmed_words:
  if word not in stop_words:
    filtered_words.append(word)

In [0]:
filtered_words

## Lemmatizing with appropriate POS tag

In [0]:
nltk.download('wordnet')                        # Lemmatization
nltk.download('averaged_perceptron_tagger')     # POS tagging

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [0]:
lem_words = []
lem = WordNetLemmatizer()
lem_words = [lem.lemmatize(word, get_wordnet_pos(word)) for word in filtered_word if word]

In [0]:
lem_words[-10:]

['suppose',
 'speak',
 'husband',
 'see',
 'choose',
 'date',
 'come',
 'back',
 'you thanks',
 'help']

## NER

In [0]:
nlp = en_core_web_sm.load()

In [0]:
text = ' '.join(lem_words)

In [0]:
convo_nlp = nlp(text)

In [0]:
labels = [ent.label_ for ent in convo_nlp.ents]
Counter(labels)

Counter({'CARDINAL': 1, 'DATE': 4, 'PRODUCT': 2})

In [0]:
#Print a random sentence from these conversations 
sentences = [x for x in convo_nlp.sents]
print(sentences[0])

like book trip atlantis caprica saturday august 13 2016 8 adult tight budget 1700 hi checked option unfortunately currently trip meet criterion like book alternate travel option yes go neverland caprica august 13 2016 5 adult trip budget 1900 checked


In [0]:
displacy.render(convo_nlp, jupyter=True, style='ent', page=True)

# Chatterbot

In [0]:
# Create a new chat bot named Charlie
chatbot = ChatBot('Charlie')
trainer = ListTrainer(chatbot)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
trainer.train([
    'How are you?',
    'I am good.',
    'That is good to hear.',
    'Thank you',
    'You are welcome.',
])

List Trainer: [####################] 100%


In [0]:
trainer.train(flat_list)

List Trainer: [####################] 100%


In [0]:
print(chatbot.get_response('Hi'))

To start, just give me some information on where you would like to travel, your budget, your point of departure, or any other travel info you can think of.


In [0]:
print(chatbot.get_response('I would like to travel from Toronto to Japan'))

Did you have any specific dates in mind?


In [0]:
print(chatbot.get_response('June to August'))

How much would that be?


In [0]:
print(chatbot.get_response('under 1200'))

Perfect. Thank you.
